In [1]:
import polars as pl

In [2]:
df = pl.read_csv("ingested.csv")
df = df.with_columns(
    (pl.col("price").ge(1000000).cast(pl.Int64).alias("ge_1M"))
)
display(df.sample())
print(list(df.columns))

address,lease_remaining,minimum_floor,maximum_floor,built_year,sqft,sqm,flat_type,road,town,year_of_sale,month_of_sale,months_ago,price,psf,band_name,price_is_geq_one_million,ge_1M
str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,i64,f64,str,i64,i64
"""627 JURONG WEST STREET 65""",76,10,12,2001,1399,130,"""EXECUTIVE""","""JURONG WEST STREET 65""","""JURONG WEST""",2018,9,75,565000,403.85,"""75 - 79""",0,0


['address', 'lease_remaining', 'minimum_floor', 'maximum_floor', 'built_year', 'sqft', 'sqm', 'flat_type', 'road', 'town', 'year_of_sale', 'month_of_sale', 'months_ago', 'price', 'psf', 'band_name', 'price_is_geq_one_million', 'ge_1M']


In [3]:
df_filter = df.filter(
    (pl.col("year_of_sale").eq(2024) & pl.col("month_of_sale").le(11)) | 
    (pl.col("year_of_sale").eq(2023)) |
    (pl.col("year_of_sale").eq(2022) & pl.col("month_of_sale").ge(12))
)
display(
    df_filter
    .group_by("ge_1M")
    .agg(pl.col("address").len().alias("count_transactions"))
    .with_columns(
        (pl.col("count_transactions")/pl.col("count_transactions").sum()*100).alias("pct_transactions")
    ).sort("ge_1M")
)
display(
    df_filter
    .group_by("year_of_sale",)
    .agg(pl.col("address").len().alias("count_transactions"))
    .sort("year_of_sale",)
)

ge_1M,count_transactions,pct_transactions
i64,u32,f64
0,52292,97.325467
1,1437,2.674533


year_of_sale,count_transactions
i64,u32
2022,2234
2023,25755
2024,25740


In [4]:
df_filter = df.filter(
    (pl.col("year_of_sale").eq(2024) & pl.col("month_of_sale").le(11))
)
display(
    df_filter
    .group_by("ge_1M")
    .agg(pl.col("address").len().alias("count_transactions"))
    .with_columns(
        (pl.col("count_transactions")/pl.col("count_transactions").sum()*100).alias("pct_transactions")
    )
)

ge_1M,count_transactions,pct_transactions
i64,u32,f64
0,24800,96.348096
1,940,3.651904


In [5]:
df_filter = df.filter(
    (pl.col("year_of_sale").eq(2023))
)
display(
    df_filter
    .group_by("ge_1M")
    .agg(pl.col("address").len().alias("count_transactions"))
    .with_columns(
        (pl.col("count_transactions")/pl.col("count_transactions").sum()*100).alias("pct_transactions")
    )
)

ge_1M,count_transactions,pct_transactions
i64,u32,f64
1,469,1.821006
0,25286,98.178994


In [6]:
df_filter = (
    df.filter(
        (pl.col("year_of_sale").eq(2024) & pl.col("month_of_sale").le(11)) | 
        (pl.col("year_of_sale").eq(2023)) |
        (pl.col("year_of_sale").eq(2022) & pl.col("month_of_sale").ge(12))
    ).with_columns(
        (pl.col("flat_type").eq("4 ROOM") & pl.col("ge_1M")).cast(pl.Int64).alias("4rm_and_ge_1M")
    )
)
display(
    df_filter
    .group_by("4rm_and_ge_1M")
    .agg(pl.col("address").len().alias("count_transactions"))
    .with_columns(
        (pl.col("count_transactions")/pl.col("count_transactions").sum()*100).alias("pct_transactions")
    ).sort("4rm_and_ge_1M")
)

4rm_and_ge_1M,count_transactions,pct_transactions
i64,u32,f64
0,53269,99.143852
1,460,0.856148
